In [2]:
# Importa pandas para manipulación de datos y FeatureStore de Feast para consumir features.
import pandas as pd
from feast import FeatureStore

/home/edyn/Documentos/obsidian-doc/Projects/MachineLearningEngineer/Curso2/Proyecto/Repo/ml2_class4_mlsupervisadoavanzado/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
# Inicializa el FeatureStore apuntando al repositorio donde están definidas las entidades, vistas y servicios de Feast.
fs = FeatureStore("../feature_repo/feature_repo")


# Consumir features offline

In [4]:
entity_df_offline = pd.DataFrame.from_dict(
        {
            # IDs de reservas para consultar features.
            "booking_id": ["f1ccaa85-7747-4dc3-be4d-5f229a2e07a4", "1466b87e-16c3-484e-b122-c0de8f375e0e"],
        },
    )

entity_df_offline["event_timestamp"] = pd.to_datetime("now", utc=True)
entity_df_offline


,booking_id,event_timestamp
0,f1ccaa85-7747-4dc3-be4d-5f229a2e07a4,2025-07-21 15:43:00.314856+00:00
1,1466b87e-16c3-484e-b122-c0de8f375e0e,2025-07-21 15:43:00.314856+00:00


In [5]:
fs.get_historical_features(
    # Consulta las features históricas en Feast usando los valores del DataFrame.
    entity_df=entity_df_offline,
    features=[
        # Solicita las features 'great_feature1' y 'great_feature2' de la vista 'pc_booking_view',
        "pc_booking_view:great_feature1",
        "pc_booking_view:great_feature2",
    ],
# Convierte el resultado a un DataFrame de pandas.
).to_df()

,booking_id,event_timestamp,great_feature1,great_feature2
0,f1ccaa85-7747-4dc3-be4d-5f229a2e07a4,2025-07-21 15:43:00.314856+00:00,2.169662,0.890985
1,1466b87e-16c3-484e-b122-c0de8f375e0e,2025-07-21 15:43:00.314856+00:00,2.309743,-0.817679


# Consumir features online

In [6]:
# Crea un DataFrame con los ids de las entidades y variables de entrada usadas en la FeatureView ondemand.
entity_df_online = pd.DataFrame.from_dict(
        {
            # IDs de reservas para consultar features.
            "booking_id": ["f1ccaa85-7747-4dc3-be4d-5f229a2e07a4", "1466b87e-16c3-484e-b122-c0de8f375e0e"],
            "kpi1": [2, 5],
            "kpi2": [10, 10 ],
        },
    )

# Añade una columna de timestamp actual en formato UTC, necesaria para la consulta histórica en Feast.
entity_df_online["event_timestamp"] = pd.to_datetime("now", utc=True)

# Muestra el DataFrame creado.
entity_df_online


,booking_id,kpi1,kpi2,event_timestamp
0,f1ccaa85-7747-4dc3-be4d-5f229a2e07a4,2,10,2025-07-21 15:43:00.733816+00:00
1,1466b87e-16c3-484e-b122-c0de8f375e0e,5,10,2025-07-21 15:43:00.733816+00:00


In [7]:
fs.get_historical_features(
    entity_df=entity_df_online,
    features=[
        "pc_booking_view:great_feature1",
        "pc_booking_view:great_feature2",
        # Solicita las features calculadas 'great_feature1_kpi1' y 'great_feature2_kpi2' de la vista on-demand 'great_feature_view'.
        "great_feature_view:great_feature1_kpi1",
        "great_feature_view:great_feature2_kpi2",
    ],
).to_df()

,booking_id,kpi1,kpi2,event_timestamp,great_feature1,great_feature2,great_feature1_kpi1,great_feature2_kpi2
0,f1ccaa85-7747-4dc3-be4d-5f229a2e07a4,2,10,2025-07-21 15:43:00.733816+00:00,2.169662,0.890985,4.339325,8.909850
1,1466b87e-16c3-484e-b122-c0de8f375e0e,5,10,2025-07-21 15:43:00.733816+00:00,2.309743,-0.817679,11.548715,-8.176788
